# STA 220 Data & Web Technologies for Data Analysis

### Lecture 10, 2/6/25, Natural language processing - LDA


### Announcements 

- One-time OH this Friday, 3-4 PM, MSB 1143
- Note the general new OH for T, 4-5, MSB 1143

### Today's topics
- Latent Dirichlet Allocation

### Latent Dirichlet Allocation

Latent Dirichlet Allocation (LDA) is a probabilistic model for a collection of discrete data. It is motivated to analyise texts. The discrete data consiststs on *words*, that are contain in *documents*, which in turn are collected in the *corpus*. Each document contains differents *topics*, which we are interested in. The topics are not observable, i.e., latent, and have to be estimated. 

- *word* is the smallest entity (token, term) of the discrete data and an element from the dictionary of $D$ words. The $i$-th word of the dictionary is a vector with zero entries except on position $i$. 
- *documents* are a series of $N$ words, denoted as $W = (w_1, \dots, w_N)$. 
- *corpus* is the collection of $M$ documents, $\{W_1, \dots, W_M\}$. 

### References
 - [Latent Dirichlet Analysis](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)

A LDA is modelled via a Bayesian hierarchy. Before we state it, we have to restate some distributions. 

##### Multinomial

Let $n,k\in\{0,1,2,...\}$ and $p = (p_1, \dots, p_k)'\in[0,1]^k$ with $\sum_{i=1}^kp_i=1$. $X\sim Mult(n,p)$ if 
$$
P(X_1=x_1, \dots, X_k = x_k) = \begin{cases} 
\frac{n!}{x_1\cdots x_k!}p_1^{x_1}\cdots p_k^{x_k}& \text{if}\sum_{i=1}^kx_i = n,\\
0, &\text{else.}
\end{cases}
$$
The Multinomial distribution is a generalization of the Binomial distribution for the case of more than two results. 

##### Dirichlet 

The Dirichlet distribution is a generalization of the Beta distribution. 
A Dirichlet distribution of order $k\geq2$ and parameters $\alpha_1, \dots, \alpha_k>0$ has the p.d.f. 
$$
f(p_1,\dots, p_k) = \frac{\Gamma(\sum_{i=1}^k\alpha_i)}{\prod_{i=1}^k\Gamma(\alpha_i)}\prod_{i=1}^kp_i^{\alpha_i-1}
$$
for all $p_1, \dots, p_k\geq0$ with $\sum_{i=1}^kp_i=1$. This implies that the $p_i$ lie on a $k-1$-dimensional simplex. For $k=2$, the Dirichlet distribution coincides with the Beta distribution. 

An important propery of the Dirichlet distribution is that its a conjugate prior to the Multiomial distribution. Consider the following hierarchy: 
\begin{align}
X|p&\sim Mult(n,p)\\
p&\sim Dir(\alpha)
\end{align}
The parameter $p$ is not assumed to be fixed (as in an frequentist understanding) but random. Consequently, it cannot be estimated. The Bayesian methodology aims in computing the posterior distribution of $p$, given the observed data $X$:
\begin{align}
P(p|X) = \frac{P(X|p)P(p)}{P(X)}
\end{align}
One can show that the posterior probabiliy is again a Dirichlet distribution!

Back to the data! We assume that every document is a collection of latent topics. Each topic is determined by the distribution of words in the document. For each document, we assume the data generating process: 

- Choose $\theta\sim Dir(\alpha)$, where $\theta, \alpha\in\mathbb{R}^k$. 
- For each word $w_i$ in the document $W$, $i = 1, \dots, N$: 
    - Choose a topic $z_i\sim Mult(1,\theta)$,
    - Choose a word from $P(w_i|z_i,\beta)$. 
        
Here, $P(w_i|z_i,\beta)$ is a multinomial probability with $n=1$, given topic $z_i$. We assume that the number of topics $k$ is known and will not be modelled as random for now. The parameter $\beta\in\mathbb{R}^{k\times D}$, and $\beta_{ij}$ is the probability that word $j$ is chosen for topic $i$.

##### Example

Consider as corpus a magazine about housekeeping. This magazine consists of three articles (documents), and each article consists of the topics `home`, `garden`, `cooking` and the words `pan`, `plot`, `window` and `way`. 

In [ ]:
import numpy as np

In [ ]:
topics = np.array(['home', 'garden', 'cooking'])
dictionary = np.array(['pan', 'tree', 'window',  'way'])

For parameters $\alpha_i$, the parameter $\theta$ determines the probability of topics in each document. 

In [ ]:
alpha = [1, 2, 3]
theta = np.random.dirichlet(alpha)
theta 

In [ ]:
zi = np.random.multinomial(1, theta) # topic for word 
zi

In [ ]:
beta = np.array([[0.1, 0.02, 0.88, 0],[0.01, 0.79, 0.1, 0.1],[0.75, 0.15, 0.1, 0]])
beta

In [ ]:
beta[zi==1,][0]

In [ ]:
np.random.multinomial(1,beta[zi==1,][0]) 

In [ ]:
zi = np.random.multinomial(1, theta) # topic for word 
wi = np.random.multinomial(1, beta[zi==1,][0]) 

print({'topic': topics[zi==1][0]})
print({'word': dictionary[wi==1][0]})

The complete corresponding hierarchy is given as 
\begin{align}
    w_i|z_i,\beta_i&\sim Mult(1,\beta_i)\\
    z_i|\theta &\sim Mult(1,\theta)\\
    \theta&\sim Dir(\alpha)
\end{align}

If $\beta_i$ and $\alpha$ are known, the posterior probability of $\theta, z_i|w_i, \alpha, \beta_i$ could be approximated using MCMC methods. The method of empirical Bayes uses estimates of $\beta_i$ and $\alpha$ to do so:  

Note that the joint distribution for a single word is given by 
$$P(\theta, z_i, w_i|\alpha_i, \beta_i) = P(w_i| z_i, \beta_i)P(z_i|\theta)P(\theta|\alpha), $$
and for a single document (let $Z = (z_1, \dots, z_N)$) thus by 
$$P(\theta, Z,W|\alpha, \beta) = \prod_{i=1}^N P(w_i| z_i, \beta_i) = P(\theta|\alpha) \prod_{i=1}^N P(w_i| z_i, \beta_i)P(z_i|\theta)$$

Integrating over $\theta$ and summing over $Z$ gives the marginal distribution over this document: 
$$
P(W|\alpha, \beta) 
= 
\int P(\theta|\alpha) \prod_{i=1}^N \sum_{z_i}P(w_i| z_i, \beta_i)P(z_i|\theta) d\theta
$$

The marginal distribution for the corpus $C$ is thus given as 
$$
P(C|\alpha, \beta) 
= 
\prod_{j=1}^M \int P(\theta|\alpha) \prod_{i=1}^N \sum_{z_{ij}}P(w_i| z_{ij}, \beta_i)P(z_{ij}|\theta) d\theta
$$

Natural estimators for $\alpha$ and $\beta$ are found by solving 
$$
\max_{\alpha, \beta} P(C|\alpha, \beta)
$$

![LDAnormal](./source/LDAnormal.png)

Problematically, if the vocabulary is large enough, it is likely of encountering a new word. Using EB, those words would receive probability zero. The usual approach in such cases is to smooth the underlying probability distribution for the words: 

\begin{align}
    w_i|z_i,\beta_i&\sim Mult(1,\beta_i)\\
    \beta_i|\eta &\sim Dir(\eta)\\
    z_i|\theta &\sim Mult(1,\theta)\\
    \theta&\sim Dir(\alpha)
\end{align}

The hyperparameters $\alpha$ and $\eta$ are estimated using EB as outlined above. 

![LDAnormal](./source/LDAsmooth.png)

#### [&#128011;](https://www.gutenberg.org/files/2701/2701-h/2701-h.htm)

In [ ]:
import nltk
import re

In [ ]:
moby = nltk.corpus.gutenberg.raw("melville-moby_dick.txt")
pattern = r"\s*(?:EXTRACTS|ETYMOLOGY\.|Epilogue|CHAPTER \d+)\s+.+\n*.+[\.!\?\)]\s*"
corpus = re.split(pattern, moby)
corpus.pop(0)
corpus = [re.sub(r"\s+", " ", document).lower() for document in corpus]

In [ ]:
len(corpus)

In [ ]:
corpus[26][:300]

From [here](https://digitalcommons.cwu.edu/cgi/viewcontent.cgi?article=1430&context=etd#:~:text=In%20Chapters%201%2D8%2C%2010,role%20of%20%22I%22%20narrator.). 

In [ ]:
cetological = [0, 1, 46, 54, 59, 60, 61, 63, 64, 68, 75, 76, 77, 78, 80, 81, 85, 86, 87, 89, 90, 91, 93, 96, 97, 98, 102, 103, 104, 105, 106]

In [ ]:
story = [i not in cetological for i in range(138)]
labels = [i for i in map(lambda x: 'story' if x else 'ceto', story)]

In [ ]:
labels[130:]

In [ ]:
corpus_tokenized = [nltk.word_tokenize(document) for document in corpus]

In [ ]:
corpus_tokenized[2][:10]

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords.extend([',', '.', ':', '!', ';', '?', '--', '\'', '\'\''])

In [ ]:
corpus_tokenized = [[word for word in document if word not in stopwords] for document in corpus_tokenized] # remove stopwords

In [ ]:
corpus_tokenized = [[nltk.PorterStemmer().stem(word) for word in document] for document in corpus_tokenized]

In [ ]:
corpus = [' '.join(document) for document in corpus_tokenized]

In [ ]:
corpus[30][:200]

In [ ]:
len(corpus)

Now, we will use the [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). 

We will have to reshape our corpus into a sparse matrix.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer(tokenizer = nltk.word_tokenize)
freq = vec.fit_transform(corpus)
corpus_freq = freq.todense() # 

In [ ]:
type(corpus_freq)

In [ ]:
corpus_freq = np.array(corpus_freq) # removes warning further down
corpus_freq

In [ ]:
corpus_freq.shape # some words less than when checked last time, due to stopword removal

In [ ]:
vec.get_feature_names_out()

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
ntopics = 2
lda = LatentDirichletAllocation(n_components = ntopics, 
                                learning_method = 'online')

In [ ]:
lda.fit(corpus_freq)

Now what? Lets investigate if the topic distribution changes over the chapters. 

In [ ]:
posterior = lda.transform(corpus_freq)
posterior 

In [ ]:
posterior.shape

In [ ]:
posterior[30]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(posterior).reset_index()
df.columns = ['chapter'] + ['Topic ' + str(i + 1) for i in range(0,ntopics)]
df

In [ ]:
ddf = pd.melt(df, id_vars = 'chapter')
ddf.head(3)

In [ ]:
import plotly.express as px
fig = px.line(ddf, x="chapter", y="value", color='variable', labels={
                     "chapter": "Chapter",
                     "value": "Probability",
                     "variable": "LDA Topics"
                 }, title = 'LDA Probabilities: Labelled cetology chapters are shaded')

for i, e in enumerate(labels): 
    if e=='ceto': 
        fig.add_vrect(x0=i-0.5, x1=i+0.5, line_width=0, fillcolor="grey", opacity=0.2)

fig.show()

In [ ]:
#df[(df['Topic 1']>0.335).values] # chapters of topic 1 - change cutoff

In [ ]:
lda.components_

In [ ]:
len(lda.components_[0])

In [ ]:
wordTopics = pd.DataFrame(lda.components_.T, index = vec.get_feature_names_out())
wordTopics

In [ ]:
wordTopics = wordTopics.apply(lambda x: x / sum(x), 1)
wordTopics.columns = ['Topic ' + str(i + 1) for i in range(0,ntopics)]
wordTopics

In [ ]:
wordTopics['Topic 1'].sort_values(ascending = False).head(10)

In [ ]:
wordTopics['Topic 2'].sort_values(ascending = False).head(10)

In [ ]:
wordTopics['Topic 3'].sort_values(ascending = False).head(10)

In [ ]:
wordTopics['Topic 4'].sort_values(ascending = False).head(10)